In [1]:
import networkcommons as nc
import pandas as pd
import networkx as nx

/home/victo/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nc.get_available_datasets()

['decryptm', 'panacea']

In [3]:
panacea_metadata, panacea_countdata = nc.download_dataset('panacea')

In [4]:
panacea_metadata

,sample_ID,group
0,ASPC_DMSO__24,ASPC_DMSO
1,ASPC_DMSO__24.1,ASPC_DMSO
2,ASPC_DMSO__24.2,ASPC_DMSO
3,ASPC_DMSO__24.3,ASPC_DMSO
4,ASPC_DMSO__24.4,ASPC_DMSO
...,...,...
1211,U87_DMSO_0_24.19,U87_DMSO
1212,U87_DMSO_0_24.20,U87_DMSO
1213,U87_DMSO_0_24.21,U87_DMSO
1214,U87_DMSO_0_24.22,U87_DMSO


In [ ]:
singlec_metadata = panacea_metadata[panacea_metadata['group'] == 'ASPC_DMSO' | panacea_metadata['group'] == 'ASPC_AFATINIB']
singlec_samples = singlec_metadata['sample_ID'].tolist()
singlec_countdata = panacea_countdata[singlec_samples in panacea_countdata.columns]

In [5]:
nc.deseq2_analysis(panacea_countdata, panacea_metadata)

AttributeError: module 'rpy2.robjects' has no attribute 'levels'

In [7]:
panacea_countdata

,gene_symbol,ASPC_DMSO__24,ASPC_DMSO__24.1,ASPC_DMSO__24.2,ASPC_DMSO__24.3,ASPC_DMSO__24.4,ASPC_DMSO__24.5,ASPC_DMSO_0_24,ASPC_ICOTINIB_7.5_24,ASPC_DMSO_0_24.1,...,U87_DMSO__24.2,U87_DMSO__24.3,U87_VARLITINIB_5.775_24.1,U87_QUIZARTINIB_0.577_24.1,U87_DMSO_0_24.18,U87_DMSO_0_24.19,U87_DMSO_0_24.20,U87_DMSO_0_24.21,U87_DMSO_0_24.22,U87_DMSO_0_24.23
0,A1BG,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
1,NAT2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ADA,6,6,9,19,7,7,1,2,6,...,4,2,0,2,2,8,5,3,4,0
3,CDH2,0,2,0,0,0,0,0,0,0,...,27,15,27,10,3,22,9,9,6,8
4,AKT3,0,0,0,0,0,0,0,0,0,...,14,5,11,22,5,18,12,8,3,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24956,PTBP3,20,39,19,20,20,19,8,25,5,...,14,26,7,15,8,13,7,4,11,4
24957,KCNE2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
24958,DGCR2,36,14,21,19,32,28,7,9,13,...,5,6,7,10,7,9,17,6,9,8
24959,CASP8AP2,37,38,11,8,6,20,13,7,3,...,16,17,13,27,6,10,14,6,9,6


In [2]:
network_datafile = 'data/network_collectri.sif'
network = nc.read_network_from_file(network_datafile)

In [3]:
source_df = pd.DataFrame({'source': ['EGFR', 'EGF'], 
                          'sign': [1, 1]}, columns=['source', 'sign'])

target_df = pd.DataFrame({'source': ['EGFR', 'EGFR', 'EGF', 'EGF'], 
                          'target': ['MAPKAPK2', 'MAPK1', 'MAPKAPK2', 'MAPK1'], 
                          'sign': [1,-1,-1,1]}, columns=['source', 'target', 'sign'])

In [4]:
shortest_path_network, shortest_paths_list = nc.shortest_paths(network, source_df, target_df)
shortest_paths_list

[['EGF', 'EGFR', 'MAP2K1', 'MAPK1'],
 ['EGF', 'EGFR', 'MAP2K1', 'MAPK3', 'MAPKAPK2'],
 ['EGF', 'EGFR', 'MAP2K1', 'MAPK1', 'MAPKAPK2'],
 ['EGFR', 'MAP2K1', 'MAPK1'],
 ['EGFR', 'MAP2K1', 'MAPK3', 'MAPKAPK2'],
 ['EGFR', 'MAP2K1', 'MAPK1', 'MAPKAPK2']]

In [5]:
shortest_sc_network, shortest_sc_list = nc.sign_consistency(shortest_path_network, shortest_paths_list, source_df, target_df)
shortest_sc_list

[['EGF', 'EGFR', 'MAP2K1', 'MAPK1'],
 ['EGFR', 'MAP2K1', 'MAPK3', 'MAPKAPK2'],
 ['EGFR', 'MAP2K1', 'MAPK1', 'MAPKAPK2']]

In [6]:
shortest_sc_network

In [7]:
all_paths_network, all_paths_list = nc.all_paths(network, source_df, target_df, depth_cutoff=4)

In [8]:
all_paths_list

[['EGF', 'EGFR', 'GRB2', 'MAP4K1', 'MAPK1'],
 ['EGF', 'EGFR', 'CRK', 'MAP4K1', 'MAPK1'],
 ['EGF', 'EGFR', 'MAP2K1', 'MAPK1'],
 ['EGF', 'ERBB2', 'GRB2', 'MAP4K1', 'MAPK1'],
 ['EGF', 'ERBB2', 'CDK1', 'PPP1CA', 'MAPK1'],
 ['EGF', 'ERBB2', 'CDK1', 'MAP2K1', 'MAPK1'],
 ['EGF', 'ERBB2', 'PAK1', 'MAP2K1', 'MAPK1'],
 ['EGF', 'ERBB2', 'PAK1', 'MAP3K1', 'MAPK1'],
 ['EGF', 'ERBB2', 'PAK1', 'ARAF', 'MAPK1'],
 ['EGF', 'ERBB2', 'PIK3CA', 'VAV1', 'MAPK1'],
 ['EGF', 'EGFR', 'MAP2K1', 'MAPK3', 'MAPKAPK2'],
 ['EGF', 'EGFR', 'MAP2K1', 'MAPK1', 'MAPKAPK2'],
 ['EGFR', 'GRB2', 'PAK1', 'MAP2K1', 'MAPK1'],
 ['EGFR', 'GRB2', 'PAK1', 'MAP3K1', 'MAPK1'],
 ['EGFR', 'GRB2', 'PAK1', 'ARAF', 'MAPK1'],
 ['EGFR', 'GRB2', 'MAP4K1', 'MAP3K1', 'MAPK1'],
 ['EGFR', 'GRB2', 'MAP4K1', 'MAPK1'],
 ['EGFR', 'JAK1', 'SYK', 'DUSP3', 'MAPK1'],
 ['EGFR', 'JAK1', 'SYK', 'MAP4K1', 'MAPK1'],
 ['EGFR', 'JAK1', 'SYK', 'PRKCA', 'MAPK1'],
 ['EGFR', 'JAK1', 'TYK2', 'DUSP3', 'MAPK1'],
 ['EGFR', 'VAV2', 'RAC1', 'MAP3K1', 'MAPK1'],
 ['EGFR', 

In [9]:
allpaths_sc_network, allpaths_sc_list = nc.sign_consistency(all_paths_network, all_paths_list, source_df, target_df)
allpaths_sc_list

[['EGF', 'EGFR', 'GRB2', 'MAP4K1', 'MAPK1'],
 ['EGF', 'EGFR', 'CRK', 'MAP4K1', 'MAPK1'],
 ['EGF', 'EGFR', 'MAP2K1', 'MAPK1'],
 ['EGF', 'ERBB2', 'GRB2', 'MAP4K1', 'MAPK1'],
 ['EGF', 'ERBB2', 'CDK1', 'PPP1CA', 'MAPK1'],
 ['EGF', 'ERBB2', 'CDK1', 'MAP2K1', 'MAPK1'],
 ['EGF', 'ERBB2', 'PAK1', 'MAP2K1', 'MAPK1'],
 ['EGF', 'ERBB2', 'PAK1', 'MAP3K1', 'MAPK1'],
 ['EGF', 'ERBB2', 'PAK1', 'ARAF', 'MAPK1'],
 ['EGF', 'ERBB2', 'PIK3CA', 'VAV1', 'MAPK1'],
 ['EGFR', 'GRB2', 'MAP4K1', 'MAPK1', 'MAPKAPK2'],
 ['EGFR', 'CRK', 'MAP4K1', 'MAPK1', 'MAPKAPK2'],
 ['EGFR', 'MAP2K1', 'MAPK3', 'MAPKAPK2'],
 ['EGFR', 'MAP2K1', 'MAPK1', 'MAPKAPK2']]

In [10]:
ppr_network = nc.add_pagerank_scores(network, source_df, target_df, personalize_for='source')
ppr_network = nc.add_pagerank_scores(ppr_network, source_df, target_df, personalize_for='target')

ppr_network = nc.compute_ppr_overlap(ppr_network)

In [16]:
nx.to_pandas_edgelist(ppr_network)

,source,target,sign,weight
0,SNCA,VAMP2,-1,1
1,DUSP7,MAPK14,-1,1
2,DUSP7,MAPK1,-1,1
3,DUSP7,GHR,-1,1
4,CD40,TRAF3,1,1
...,...,...,...,...
6146,PTPRR,STAT3,-1,1
6147,PTPRR,PXN,-1,1
6148,PTPRR,MAPK7,-1,1
6149,ERRFI1,ERBB2,-1,1
